# Package

In [12]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import shap
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr
from sklearn.utils import shuffle
import pickle


# Importation des données

In [13]:
df_stationary_train = pd.read_csv("df_stationary_train.csv", index_col="date")
df_stationary_test = pd.read_csv("df_stationary_test.csv", index_col="date")

In [14]:
df_stationary_train.head()

,UNRATE,TB3MS,RPI,INDPRO,DPCERA3M086SBEA,S&P 500,BUSLOANS,CPIAUCSL,OILPRICEx,M2SL,USREC
date,,,,,,,,,,,
1960-01-01,-0.8,0.30,0.020977,0.091980,0.001204,0.017909,0.011578,-0.006156,0.0,0.001323,0
1960-02-01,-1.1,-0.19,0.014565,0.076964,0.006009,-0.025663,0.011905,-0.003767,0.0,0.002007,0
1960-03-01,-0.2,-1.18,0.006250,0.007961,0.021240,-0.070857,-0.008356,-0.005455,0.0,0.001324,0
1960-04-01,0.0,-1.12,0.006489,-0.025915,0.033752,-0.040442,-0.009098,0.005090,0.0,0.000634,1
1960-05-01,0.0,-0.67,0.007747,-0.018121,0.009040,-0.010090,-0.000359,0.003383,0.0,0.003977,1


In [33]:
df_stationary_train.index = pd.to_datetime(df_stationary_train.index)

# stationary

In [34]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# ==========================================================
# 🔧 Hypothèse : df_stationary_train contient la cible 'Y'
# ==========================================================

# --- Définir les features
features = [c for c in df_stationary_train.columns if c != "UNRATE"]

# === Paramètres ===
step_size = 12
winsor_level = 0.01
norm_var = True

# === Conteneurs ===
models = []
coefs = []
train_periods = []

In [35]:
range(step_size, len(df_stationary_train), step_size)

range(12, 360, 12)

In [36]:
a = range(1, 5 ,1)
print(a)

range(1, 5)


In [37]:
for t, end in enumerate(a):
    #print(t)
    print(end)

1
2
3
4


In [38]:
df_stationary_train

,UNRATE,TB3MS,RPI,INDPRO,DPCERA3M086SBEA,S&P 500,BUSLOANS,CPIAUCSL,OILPRICEx,M2SL,USREC
date,,,,,,,,,,,
1960-01-01,-0.8,0.30,0.020977,0.091980,0.001204,0.017909,0.011578,-0.006156,0.000000,0.001323,0
1960-02-01,-1.1,-0.19,0.014565,0.076964,0.006009,-0.025663,0.011905,-0.003767,0.000000,0.002007,0
1960-03-01,-0.2,-1.18,0.006250,0.007961,0.021240,-0.070857,-0.008356,-0.005455,0.000000,0.001324,0
1960-04-01,0.0,-1.12,0.006489,-0.025915,0.033752,-0.040442,-0.009098,0.005090,0.000000,0.000634,1
1960-05-01,0.0,-0.67,0.007747,-0.018121,0.009040,-0.010090,-0.000359,0.003383,0.000000,0.003977,1
...,...,...,...,...,...,...,...,...,...,...,...
1989-08-01,-0.4,-0.53,0.008277,-0.000039,0.014490,0.099097,0.005449,-0.010676,-0.078372,0.014176,0
1989-09-01,-0.1,-0.40,0.008246,-0.003350,0.010644,0.070372,0.007701,-0.009804,-0.020759,0.011753,0
1989-10-01,-0.1,-0.24,0.009738,0.005091,0.007003,0.045643,-0.005351,-0.004106,0.022500,0.004694,0


# 

In [39]:
# ===============================
# Paramètres principaux
# ===============================
h = 24                 # (utilisé uniquement si vous voulez aussi le point à +24 ailleurs)
window_len = 12        # entraînement sur 12 mois
step_size = 12         # refit annuel
lag = 12               # "comme eux" : le bloc commence vers +lag
forecast_lag = lag
# winsor_level, norm_var doivent déjà exister (ex: winsor_level=0.01; norm_var=True)

In [40]:
# ===============================
# TRAIN global (1960–1989)
# ===============================
df_train_global = df_stationary_train.loc["1960-01":"1989-12"].copy()

# Exclure indicatrices du prétraitement
exclude = [c for c in ["USREC"] if c in features]
cols_tx = [c for c in features if c not in exclude]

In [41]:
# Fenêtres admissibles (rolling 12 mois), on garantit que la fin de fenêtre existe.
idx = df_train_global.index
start_end_ix = window_len - 1                          # fin de la 1re fenêtre: index 11 (0..11)
# pour l'évaluation "comme eux", on coupera les indices hors plage à la volée (bornage),
# donc on peut aller jusqu'à la fin du TRAIN global.
end_ixs = range(start_end_ix, len(idx), step_size)

In [42]:
# ===============================
# Boucle TRAIN (winsor -> norm -> fit)
# + stockage des paramètres et modèles
# ===============================
models, coefs, intercepts, train_periods = [], [], [], []
prep_params = []
pred_rows_block = []   # pour l'évaluation "comme eux"

for t, end_ix in enumerate(end_ixs):
    start_ix = end_ix - window_len + 1
    df_win = df_train_global.iloc[start_ix:end_ix + 1].copy()

    X_tr = df_win[features].copy()
    y_tr = df_win["UNRATE"].copy()

    # 1) Winsorisation (apprise sur la fenêtre)
    lower_wins = X_tr[cols_tx].quantile(winsor_level)
    upper_wins = X_tr[cols_tx].quantile(1 - winsor_level)
    X_tr.loc[:, cols_tx] = X_tr[cols_tx].clip(lower=lower_wins, upper=upper_wins, axis=1)

    # 2) Normalisation (apprise sur la fenêtre)
    if norm_var:
        mean_tr = X_tr[cols_tx].mean()
        std_tr  = X_tr[cols_tx].std().replace(0, 1)
        X_tr.loc[:, cols_tx] = (X_tr[cols_tx] - mean_tr) / std_tr
    else:
        mean_tr = None
        std_tr  = None

    # 3) Fit modèle
    model = LinearRegression().fit(X_tr, y_tr)

    # 4) Sauvegardes par fenêtre
    models.append(model)
    coefs.append(pd.Series(model.coef_, index=features))
    intercepts.append(model.intercept_)
    train_periods.append(df_win.index[-1])
    prep_params.append({
        "lower_wins": lower_wins,
        "upper_wins": upper_wins,
        "mean_train": mean_tr,
        "std_train": std_tr,
        "exclude": list(exclude),
        "features": list(features),
        "window_len": window_len,
    })

    # -------- ÉVALUATION "comme eux" : bloc de step_size prédictions à partir de ~+lag --------
    # position entière de la fin de fenêtre
    end_pos = int(np.where(idx == df_win.index[-1])[0][0])
    skip_first = forecast_lag - 1
    horizon = step_size

    forecast_pos = [end_pos + skip_first + k for k in range(horizon)]
    # bornage pour rester dans la plage TRAIN
    forecast_pos = [p for p in forecast_pos if 0 <= p < len(idx)]
    if len(forecast_pos) > 0:
        X_blk = df_train_global.iloc[forecast_pos][features].copy()

        # ré-appliquer EXACTEMENT les paramètres de la fenêtre (pas de fuite)
        X_blk.loc[:, cols_tx] = X_blk[cols_tx].clip(
            lower=lower_wins, upper=upper_wins, axis=1
        )
        if norm_var and (mean_tr is not None):
            std_safe = std_tr.replace(0, 1)
            X_blk.loc[:, cols_tx] = (X_blk[cols_tx] - mean_tr) / std_safe

        y_hat_blk = model.predict(X_blk)
        y_true_blk = df_train_global.iloc[forecast_pos]["UNRATE"].values
        dates_blk = df_train_global.index[forecast_pos]

        for d, yh, yt in zip(dates_blk, y_hat_blk, y_true_blk):
            pred_rows_block.append({
                "date_forecast": d,
                "UNRATE_true": float(yt),
                "UNRATE_pred": float(yh),
                "train_end": df_win.index[-1]
            })

    print(f"[{t:02d}] fin_train={df_win.index[-1].strftime('%Y-%m')} | "
          f"bloc={'—' if len(forecast_pos)==0 else (pd.to_datetime(df_train_global.index[forecast_pos[0]]).strftime('%Y-%m') + '→' + pd.to_datetime(df_train_global.index[forecast_pos[-1]]).strftime('%Y-%m'))} "
          f"(n={len(forecast_pos)})")

[00] fin_train=1960-12 | bloc=1961-11→1962-10 (n=12)
[01] fin_train=1961-12 | bloc=1962-11→1963-10 (n=12)
[02] fin_train=1962-12 | bloc=1963-11→1964-10 (n=12)
[03] fin_train=1963-12 | bloc=1964-11→1965-10 (n=12)
[04] fin_train=1964-12 | bloc=1965-11→1966-10 (n=12)
[05] fin_train=1965-12 | bloc=1966-11→1967-10 (n=12)
[06] fin_train=1966-12 | bloc=1967-11→1968-10 (n=12)
[07] fin_train=1967-12 | bloc=1968-11→1969-10 (n=12)
[08] fin_train=1968-12 | bloc=1969-11→1970-10 (n=12)
[09] fin_train=1969-12 | bloc=1970-11→1971-10 (n=12)
[10] fin_train=1970-12 | bloc=1971-11→1972-10 (n=12)
[11] fin_train=1971-12 | bloc=1972-11→1973-10 (n=12)
[12] fin_train=1972-12 | bloc=1973-11→1974-10 (n=12)
[13] fin_train=1973-12 | bloc=1974-11→1975-10 (n=12)
[14] fin_train=1974-12 | bloc=1975-11→1976-10 (n=12)
[15] fin_train=1975-12 | bloc=1976-11→1977-10 (n=12)
[16] fin_train=1976-12 | bloc=1977-11→1978-10 (n=12)
[17] fin_train=1977-12 | bloc=1978-11→1979-10 (n=12)
[18] fin_train=1978-12 | bloc=1979-11→1980-10 

In [43]:
# ===============================
# Agrégation des prédictions "comme eux"
# ===============================
oos_train_block = (
    pd.DataFrame(pred_rows_block)
      .set_index("date_forecast")
      .sort_index()
)

print("\nAperçu oos_train_block :")
print(oos_train_block.head())
print("Plage couverte :", oos_train_block.index.min(), "→", oos_train_block.index.max())
print("Taille totale  :", len(oos_train_block))


Aperçu oos_train_block :
               UNRATE_true  UNRATE_pred  train_end
date_forecast                                     
1961-11-01             0.0    -0.061475 1960-12-01
1961-12-01            -0.6    -0.978215 1960-12-01
1962-01-01            -0.8    -0.523134 1960-12-01
1962-02-01            -1.4    -0.008573 1960-12-01
1962-03-01            -1.3     0.728959 1960-12-01
Plage couverte : 1961-11-01 00:00:00 → 1989-12-01 00:00:00
Taille totale  : 338


In [44]:
# Emballage des résultats
# ===============================
exp_results = {
    "models": models,
    "coefs": coefs,
    "intercepts": intercepts,
    "train_periods": train_periods,
    "features": list(features),
    "prep_params": prep_params,
    "pred_block_train": oos_train_block,   # prédictions "comme eux" dans le TRAIN
    "params": {
        "winsor_level": winsor_level,
        "norm_var": norm_var,
        "step_size": step_size,
        "window_len": window_len,
        "lag": lag,
        "forecast_lag_like_them": forecast_lag,
        "horizon_like_them": step_size,
        "train_span": ["1960-01", "1989-12"],
    },
}

print("\n✅ Terminé : modèles, prétraitements et bloc de prédictions (comme eux) sont prêts.")


✅ Terminé : modèles, prétraitements et bloc de prédictions (comme eux) sont prêts.


In [45]:
# ===============================
# ÉVALUATION du bloc "comme eux"
# ===============================
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr

# 1) Préparation propre
df_eval = oos_train_block.copy().sort_index()
df_eval = df_eval.replace([np.inf, -np.inf], np.nan).dropna(subset=["UNRATE_true", "UNRATE_pred"])

y_true = df_eval["UNRATE_true"].to_numpy()
y_pred = df_eval["UNRATE_pred"].to_numpy()

# 2) Métriques globales
r2   = r2_score(y_true, y_pred) if len(df_eval) > 0 else np.nan
r2_0 = r2_score(y_true, np.zeros_like(y_true)) if len(df_eval) > 0 else np.nan
mae  = mean_absolute_error(y_true, y_pred)     if len(df_eval) > 0 else np.nan
rmse = np.sqrt(mean_squared_error(y_true, y_pred)) if len(df_eval) > 0 else np.nan
corr, pval = (pearsonr(y_true, y_pred) if len(df_eval) > 1 else (np.nan, np.nan))
amd  = np.mean(np.abs(y_true - np.mean(y_true))) if len(df_eval) > 0 else np.nan

print("\n=== Évaluation TRAIN (bloc façon eux) ===")
print(f"n obs          : {len(df_eval)}")
print(f"R²             : {r2:.4f}")
print(f"R² (OLS b=0)   : {r2_0:.4f}")
print(f"MAE            : {mae:.4f}")
print(f"RMSE           : {rmse:.4f}")
print(f"Corrélation    : {corr:.4f} (p={pval:.2e})")
print(f"Abs Mean Dev.  : {amd:.4f}")


=== Évaluation TRAIN (bloc façon eux) ===
n obs          : 338
R²             : -3.3739
R² (OLS b=0)   : -0.0022
MAE            : 1.6719
RMSE           : 2.3407
Corrélation    : 0.0045 (p=9.35e-01)
Abs Mean Dev.  : 0.8114


# 2) Indices des fenêtres admissibles (respecter t+h ∈ TRAIN)

In [21]:
idx = df_train_global.index
last_trainable_ix = len(idx) - 1 - h                   # pour que end_ix + h reste dans 1960–1989
start_end_ix = window_len - 1                          # 1ère fenêtre de 12 mois (0…11)
end_ixs = range(start_end_ix, last_trainable_ix + 1, step_size)

3) Boucle TRAIN (12 mois) + prétraitements par fenêtre + fit

In [22]:
models, coefs, intercepts, train_periods = [], [], [], []
prep_params = []
pred_rows = []   # pour stocker les prévisions à h=24 (toujours dans TRAIN)

for t, end_ix in enumerate(end_ixs):
    start_ix = end_ix - window_len + 1
    df_win = df_train_global.iloc[start_ix:end_ix + 1].copy()

    X_tr = df_win[features].copy()
    y_tr = df_win["UNRATE"].copy()

    # 1) Winsorisation (bornes apprises sur la fenêtre de 12 mois)
    lower_wins = X_tr[cols_tx].quantile(winsor_level)
    upper_wins = X_tr[cols_tx].quantile(1 - winsor_level)
    X_tr.loc[:, cols_tx] = X_tr[cols_tx].clip(lower=lower_wins, upper=upper_wins, axis=1)

    # 2) Normalisation (stats apprises sur la fenêtre)
    if norm_var:
        mean_tr = X_tr[cols_tx].mean()
        std_tr  = X_tr[cols_tx].std().replace(0, 1)
        X_tr.loc[:, cols_tx] = (X_tr[cols_tx] - mean_tr) / std_tr
    else:
        mean_tr = None
        std_tr  = None

    # 3) Entraînement (12 mois)
    model = LinearRegression().fit(X_tr, y_tr)

    # 4) Prévision à h = 24 mois (toujours DANS 1960–1989)
    test_ix = end_ix + h
    X_h = df_train_global.iloc[[test_ix]][features].copy()
    # Ré-appliquer exactement les mêmes paramètres (pas de fuite)
    X_h.loc[:, cols_tx] = X_h[cols_tx].clip(lower=lower_wins, upper=upper_wins, axis=1)
    if norm_var:
        std_safe = std_tr.replace(0, 1)
        X_h.loc[:, cols_tx] = (X_h[cols_tx] - mean_tr) / std_safe

    y_hat_h = float(model.predict(X_h)[0])
    y_true_h = float(df_train_global.iloc[test_ix]["UNRATE"])
    date_h = df_train_global.index[test_ix]

    # 5) Sauvegardes
    models.append(model)
    coefs.append(pd.Series(model.coef_, index=features))
    intercepts.append(model.intercept_)
    train_periods.append(df_win.index[-1])
    prep_params.append({
        "lower_wins": lower_wins,
        "upper_wins": upper_wins,
        "mean_train": mean_tr,
        "std_train": std_tr,
        "exclude": list(exclude),
        "features": list(features),
        "window_len": window_len,
    })
    pred_rows.append({
        "date_h": date_h,
        "UNRATE_true_h": y_true_h,
        "UNRATE_pred_h": y_hat_h,
        "train_end": df_win.index[-1]
    })

    print(f"[{t:02d}] Fin {df_win.index[-1].strftime('%Y-%m')} | fit sur {len(df_win)} obs | prévision @+{h} → {pd.to_datetime(date_h).strftime('%Y-%m')}")

AttributeError: 'str' object has no attribute 'strftime'

4) Séries de prévisions à h=24 (TRAIN only)

In [ ]:
oos_train_h = pd.DataFrame(pred_rows).set_index("date_h").sort_index()

In [ ]:
# === Évaluation globale (TRAIN, horizon h=24) ===
import numpy as np
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr

y_true = oos_train_h["UNRATE_true_h"].values
y_pred = oos_train_h["UNRATE_pred_h"].values

r2 = r2_score(y_true, y_pred)
r2_null = r2_score(y_true, np.zeros_like(y_true))  # baseline b=0
mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
corr, pval = pearsonr(y_true, y_pred)
amd = np.mean(np.abs(y_true - np.mean(y_true)))

print("\n=== Évaluation TRAIN (h=24) ===")
print(f"R²              : {r2:.4f}")
print(f"R² (OLS b=0)    : {r2_null:.4f}")
print(f"MAE             : {mae:.4f}")
print(f"RMSE            : {rmse:.4f}")
print(f"Corrélation     : {corr:.4f} (p={pval:.2e})")
print(f"Abs Mean Dev.   : {amd:.4f}")


=== Évaluation TRAIN (h=24) ===
R²              : -1.5038
R² (OLS b=0)    : -0.0004
MAE             : 1.3645
RMSE            : 1.7607
Corrélation     : 0.1306 (p=5.08e-01)
Abs Mean Dev.   : 0.7954


In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr
import shap

# ==========================================================
# 📈 ÉVALUATION SUR LE TRAIN GLOBAL
# ==========================================================
features = exp_results["features"]
model_final = exp_results["models"][-1]  # dernier modèle
winsor_level = exp_results["params"]["winsor_level"]
norm_var = exp_results["params"]["norm_var"]

X_full = df_stationary_train[features].copy()
Y_full = df_stationary_train["UNRATE"].copy()

# Winsorisation + normalisation globales
lower_wins = X_full.quantile(winsor_level)
upper_wins = X_full.quantile(1 - winsor_level)
X_full = X_full.clip(lower=lower_wins, upper=upper_wins, axis=1)

if norm_var:
    mean_full = X_full.mean()
    std_full = X_full.std().replace(0, 1)
    X_full = (X_full - mean_full) / std_full

# === Prédictions et métriques
y_pred = model_final.predict(X_full)

r2 = r2_score(Y_full, y_pred)
r2_null = r2_score(Y_full, np.zeros_like(Y_full))
mae = mean_absolute_error(Y_full, y_pred)
rmse = np.sqrt(mean_squared_error(Y_full, y_pred))
corr, pval = pearsonr(Y_full, y_pred)
amd = np.mean(np.abs(Y_full - np.mean(Y_full)))

import joblib
import os

# Chemin de sauvegarde
model_dir = "models"
os.makedirs(model_dir, exist_ok=True)

# Fichier du modèle
model_path = os.path.join(model_dir, "model_final.pkl")

# Dictionnaire complet à sauvegarder
saved_model = {
    "model": model_final,
    "features": features,
    "winsor_level": winsor_level,
    "norm_var": norm_var,
    "mean_full": mean_full if norm_var else None,
    "std_full": std_full if norm_var else None,
}

# Sauvegarde avec joblib
joblib.dump(saved_model, model_path)

print(f"\n✅ Modèle sauvegardé avec succès : {model_path}")

print("\n=== Performance globale (TRAIN) ===")
print(f"R² (ensemble) : {r2:.4f}")
print(f"R² (OLS b=0)  : {r2_null:.4f}")
print(f"MAE            : {mae:.4f}")
print(f"RMSE           : {rmse:.4f}")
print(f"Corrélation    : {corr:.4f} (p = {pval:.2e})")
print(f"Abs Mean Dev.  : {amd:.4f}")


✅ Modèle sauvegardé avec succès : models\model_final.pkl

=== Performance globale (TRAIN) ===
R² (ensemble) : -0.1009
R² (OLS b=0)  : -0.0000
MAE            : 0.8792
RMSE           : 1.1751
Corrélation    : 0.5711 (p = 1.51e-32)
Abs Mean Dev.  : 0.8289


In [ ]:
# ==========================================================
# 🔍 IMPORTANCE PAR PERMUTATION
# ==========================================================
def permutation_importance(model, X, y, metric=mean_absolute_error, n_repeats=30):
    base_score = metric(y, model.predict(X))
    results = []
    for col in X.columns:
        scores = []
        for _ in range(n_repeats):
            X_shuffled = X.copy()
            X_shuffled[col] = np.random.permutation(X_shuffled[col])
            score = metric(y, model.predict(X_shuffled))
            scores.append(score)
        scores = np.array(scores)
        results.append({
            "variable": col,
            "perm_mae_ratio": np.mean(scores) / base_score,
            "perm_rmse_ratio": np.sqrt(mean_squared_error(y, model.predict(X_shuffled))) / rmse,
            "perm_deviance": np.mean(np.abs(scores - base_score))
        })
    return pd.DataFrame(results).sort_values("perm_mae_ratio", ascending=False)

perm_df = permutation_importance(model_final, X_full, Y_full)

print("\n=== Importance par permutation ===")
print(perm_df.head(10).to_string(index=False))


=== Importance par permutation ===
       variable  perm_mae_ratio  perm_rmse_ratio  perm_deviance
          USREC        1.291937         1.257301       0.188454
        S&P 500        1.127039         1.087991       0.082007
            RPI        1.095025         1.073389       0.061341
         INDPRO        1.013560         1.009293       0.008754
       BUSLOANS        1.011036         1.002059       0.007124
DPCERA3M086SBEA        1.010665         1.006124       0.007029
      OILPRICEx        1.010090         0.998821       0.006860
       CPIAUCSL        1.006380         1.002696       0.004358
          TB3MS        1.003201         1.002882       0.002403
           M2SL        0.998155         1.000433       0.001575


In [ ]:

# ==========================================================
# 💡 IMPORTANCE SHAPLEY
# ==========================================================
explainer = shap.Explainer(model_final, X_full)
shap_values = explainer(X_full)

shap_df = pd.DataFrame({
    "variable": X_full.columns,
    "shap_mean_abs": np.abs(shap_values.values).mean(axis=0),
})
shap_df["shap_share"] = shap_df["shap_mean_abs"] / shap_df["shap_mean_abs"].sum()
shap_df = shap_df.sort_values("shap_mean_abs", ascending=False)

print("\n=== Importance Shapley (shares) ===")
print(shap_df.head(10).to_string(index=False))


=== Importance Shapley (shares) ===
       variable  shap_mean_abs  shap_share
          USREC       0.340596    0.312587
        S&P 500       0.249643    0.229113
            RPI       0.185142    0.169917
         INDPRO       0.067299    0.061764
       BUSLOANS       0.052476    0.048161
DPCERA3M086SBEA       0.052252    0.047955
      OILPRICEx       0.046934    0.043074
       CPIAUCSL       0.043060    0.039519
           M2SL       0.027262    0.025020
          TB3MS       0.024941    0.022890


In [ ]:
# ==========================================================
# 🧩 COMPARATIF GLOBAL
# ==========================================================
merged_df = pd.merge(perm_df, shap_df, on="variable", how="outer")
print("\n=== Résumé comparatif (Permutation + SHAP) ===")
print(merged_df.head(10).to_string(index=False))


=== Résumé comparatif (Permutation + SHAP) ===
       variable  perm_mae_ratio  perm_rmse_ratio  perm_deviance  shap_mean_abs  shap_share
       BUSLOANS        1.011036         1.002059       0.007124       0.052476    0.048161
       CPIAUCSL        1.006380         1.002696       0.004358       0.043060    0.039519
DPCERA3M086SBEA        1.010665         1.006124       0.007029       0.052252    0.047955
         INDPRO        1.013560         1.009293       0.008754       0.067299    0.061764
           M2SL        0.998155         1.000433       0.001575       0.027262    0.025020
      OILPRICEx        1.010090         0.998821       0.006860       0.046934    0.043074
            RPI        1.095025         1.073389       0.061341       0.185142    0.169917
        S&P 500        1.127039         1.087991       0.082007       0.249643    0.229113
          TB3MS        1.003201         1.002882       0.002403       0.024941    0.022890
          USREC        1.291937         1.

# 📊 Métriques utilisées

## 1) Performance globale

**Coefficient de détermination (R²)**  
$$
R^2 = 1 - \frac{\sum_i (y_i - \hat{y}_i)^2}{\sum_i (y_i - \bar{y})^2}
$$  
➡️ Part de la variance expliquée par le modèle (0 = pas mieux que la moyenne, 1 = parfait).

**Erreur absolue moyenne (MAE)**  
$$
MAE = \frac{1}{n}\sum_{i=1}^n |y_i - \hat{y}_i|
$$  
➡️ Écart absolu moyen entre valeurs réelles et prédites, robuste aux outliers.

**Erreur quadratique moyenne (RMSE)**  
$$
RMSE = \sqrt{\frac{1}{n}\sum_{i=1}^n (y_i - \hat{y}_i)^2}
$$  
➡️ Similaire au MAE mais pénalise davantage les grosses erreurs.

**Corrélation de Pearson**  
$$
\rho(y, \hat{y}) = \frac{\text{Cov}(y, \hat{y})}{\sigma_y \cdot \sigma_{\hat{y}}}
$$  
➡️ Mesure le degré de lien linéaire entre les prédictions et les observations.

**Abs Mean Deviance (AMD)**  
$$
AMD = \frac{1}{n}\sum_{i=1}^n |\hat{y}_i - \bar{\hat{y}}|
$$  
➡️ Écart moyen des prédictions par rapport à leur moyenne ; sert de référence pour la permutation prédiction-basée.

---

## 2) Importance par permutation
La relation entre Y et X dépend du temps. Quand on perturbe la série X (en la mélangeant), on casse ce lien, et si l’erreur augmente, cela montre que X est une variable clé pour expliquer Y.

**Ratio MAE**  
$$
PI^{MAE}_j = \frac{MAE^{(perm)}_j}{MAE^{(base)}}
$$  
➡️ Si > 1, la variable est utile pour réduire l’erreur absolue.

**Ratio RMSE**  
$$
PI^{RMSE}_j = \frac{RMSE^{(perm)}_j}{RMSE^{(base)}}
$$  
➡️ Si > 1, la variable aide à limiter les grosses erreurs.

**Déviance de prédiction**  
$$
PI^{dev}_j = \frac{1}{n}\sum_{i=1}^n \big|\hat{y}_i - \hat{y}^{(perm)}_{i,j}\big|
$$  
➡️ Mesure combien les prédictions changent quand on brouille une variable.

---

## 3) Importance Shapley

**Décomposition des prédictions**  
$$
\hat{y}_i = \phi_0 + \sum_{j=1}^p \phi_{ij}
$$  
➡️ Chaque prédiction est expliquée par une contribution \(\phi_{ij}\) par variable.

**Importance absolue moyenne**  
$$
\text{Mean}(|\phi_j|) = \frac{1}{n}\sum_{i=1}^n |\phi_{ij}|
$$  
➡️ Contribution moyenne (absolue) d’une variable sur toutes les prédictions.

**Shapley share**  
$$
\Gamma_j = \frac{\text{Mean}(|\phi_j|)}{\sum_{k=1}^p \text{Mean}(|\phi_k|)}
$$  
➡️ Part relative de la variable dans l’explication totale (somme des parts = 1).

## Interprétation des résultats 

### Performance globale 
- R² = 0.2266 → modèle OLS explique ~23 % de la variance du chômage US.
- MAE = 0.6774 → en moyenne, l’erreur absolue est de 0.68 points (dans l’unité de la variable cible).
- RMSE = 0.8750 → un peu plus élevé que le MAE, ce qui indique la présence de grosses erreurs ponctuelles.
- Corrélation = 0.4760 (p ≈ 10⁻³⁵) → lien positif et significatif entre prédictions et observations, mais seulement modéré. Ce qui peut expliquer la présence d'une relation 
- Abs Mean Deviance = 0.3370 → sert ici de référence pour l’importance prédiction-basée : les prédictions s’écartent en moyenne de 0.34 de leur propre moyenne.

Lecture : le modèle OLS capte une partie utile du signal, mais laisse beaucoup de variance inexpliquée. La corrélation faible illustre éventuellement la présence des relations non-linéaire, et non captées par OLS.

### 🔹 2. Importance par permutation
- INDPRO (Industrial Production) : la plus influente. Sa permutation augmente MAE de +19 % et RMSE de +20 %, avec une forte déviance de prédiction (0.41).
- TB3MS (Taux d’intérêt à 3 mois) : impact non négligeable, ratios ~1.02 et déviance ~0.10.
- BUSLOANS (Prêts commerciaux) : rôle similaire (MAE ratio 1.017, déviance ~0.09).
- S&P 500 : contribution modérée, ratios légèrement > 1.
- RPI, M2SL : influence plus faible mais perceptible.
- CPIAUCSL, OILPRICEx, DPCERA3M086SBEA : quasi neutres (ratios ≈ 1, déviance très faible).

Lecture : INDPRO domine largement la performance, les autres apportent des compléments mais plus modestes.

### 🔹 3. Importance Shapley (shares)
- INDPRO : ~52 % de l’explication totale des prédictions → cohérence parfaite avec la permutation.
- TB3MS (12 %) + BUSLOANS (12 %) : deux autres piliers importants.
- S&P 500 (9,7 %) : contribue de façon notable.
- M2SL (5 %), RPI (3 %), CPIAUCSL (3,5 %) : apports plus secondaires.
- OILPRICEx et DPCERA3M086SBEA (<2 %) : quasi négligeables dans ce modèle.

Lecture : INDPRO est la variable macroéconomique centrale, suivie par des indicateurs financiers (taux courts, prêts bancaires, marché actions).

# Test

In [ ]:
import joblib

saved_model = joblib.load("models/model_final.pkl")
model_final = saved_model["model"]
features = saved_model["features"]
winsor_level = saved_model["winsor_level"]
norm_var = saved_model["norm_var"]
mean_full = saved_model["mean_full"]
std_full = saved_model["std_full"]